In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv',
#                        index_col=0,
                       parse_dates=['date_time'])

test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv',
#                        index_col=0,
                       parse_dates=['date_time'])

train_df

In [ ]:
train_df['hour'] = train_df['date_time'].dt.hour
test_df['hour'] = test_df['date_time'].dt.hour

train_df['day'] = train_df['date_time'].dt.day
test_df['day'] = test_df['date_time'].dt.day

train_df['week'] = train_df['date_time'].dt.dayofweek
test_df['week'] = test_df['date_time'].dt.dayofweek

train_df['month'] = train_df['date_time'].dt.month
test_df['month'] = test_df['date_time'].dt.month

train_df

In [ ]:
y = train_df.loc[ :, ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
train_df
X = train_df.drop(['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1)
test = test_df.drop(['date_time'], axis=1)
test

In [ ]:
total_df = pd.concat([X, test])
total_df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(total_df))
scaled_X = scaled_features[:X.shape[0]]
scaled_test = scaled_features[X.shape[0]:]

print('The train data shape:', scaled_X.shape)
print('The target data shape:', y.shape)
print('The test data shape:', scaled_test.shape)

In [ ]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb_model = MultiOutputRegressor(XGBRegressor(learning_rate=0.01, n_jobs=-1, n_estimators=100))
xgb_model.get_params()

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from warnings import simplefilter
simplefilter = True

# X_train, X_valid, y_train, y_valid = train_test_split(scaled_X, y)

cv_fold = KFold(n_splits=5, shuffle=True, random_state=1)

scores = (-1 * cross_val_score(xgb_model, scaled_X, y, cv=5, scoring='neg_mean_squared_log_error')) ** 0.5
scores.mean()

In [ ]:
# from hyperopt import hp, fmin, tpe, space_eval

# def objective(search_space):
    
#     model = MultiOutputRegressor(XGBRegressor(**search_space))
    
#     cv_fold = KFold(n_splits=5, shuffle=True)
    
#     scores = (-1 * cross_val_score(model, scaled_X, y, cv=5, scoring='neg_mean_absolute_error'))
    
#     return scores.mean()

# algorithm = tpe.suggest

# search_space = {
#     'n_estimators': hp.randint('n_estimators', 50, 500),
#     'learning_rate': hp.loguniform('learning_rate', np.log(0.05), np.log(0.2)),
#     'max_depth': hp.randint('max_depth', 10, 200),
#     'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
#     'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
#     'min_child_weight': hp.quniform('min_child', 1, 20, 1),
#     'subsample': hp.uniform('subsample', 0.6, 1.0),
#     'gamma': hp.quniform('gamma', 0.5, 1.0, 0.05),
#     'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05)
# }

# best_scores = fmin(
#     fn = objective,
#     space=search_space,
#     max_evals=20,
#     algo=algorithm, 
# )

# space_eval(search_space, best_scores)

# OUTPUT:

# 100%|██████████| 20/20 [33:59<00:00, 101.99s/trial, best loss: 20.183540733614088]
# {'colsample_bytree': 0.9,
#  'gamma': 0.9,
#  'learning_rate': 0.06506574124858261,
#  'max_depth': 34,
#  'min_child_weight': 17.0,
#  'n_estimators': 459,
#  'reg_alpha': 0.3595203758253189,
#  'reg_lambda': 0.9802190953463499,
#  'subsample': 0.9627465567278952}

In [ ]:
final_model = MultiOutputRegressor(XGBRegressor(
 colsample_bytree=0.9,
 gamma=0.9,
 learning_rate=0.06506574124858261,
 max_depth=34,
 min_child_weight=17.0,
 n_estimators=459,
 reg_alpha=0.3595203758253189,
 reg_lambda=0.9802190953463499,
 subsample=0.9627465567278952
))

final_model.fit(scaled_X, y)

y_pred = final_model.predict(scaled_test)

In [ ]:
submission = pd.DataFrame(y_pred, columns=['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], index=test_df['date_time'])
submission.to_csv('submission.csv')